In [30]:
import glob
import pandas as pd
from geopy.geocoders import Nominatim

In [31]:
loc = pd.read_csv('all_locations.csv')
df = pd.read_csv('Master_checklist.csv')

In [32]:
loc['Locations'] = loc['Locations'].str.strip()

In [33]:
loc = dict(zip(loc['Locations'], zip(loc['Lat'], loc['Long'])))

In [34]:
loc

{'Denver': (39.73, -104.99),
 'New Jersey': (40.14, -74.67),
 'Washington DC': (38.89, -77.03),
 'Seattle': (47.6, -122.32),
 'San Fran': (37.73, -122.46),
 'New york': (42.93, -75.61),
 'Santa Fe': (35.69, -105.93),
 'Dallas': (32.77, -96.79),
 'Jackson': (43.47, -110.76),
 'Portland': (45.51, -122.67),
 'Los Angeles': (34.05, -118.24)}

In [35]:
df.drop(columns='Unnamed: 0', inplace=True)

In [36]:
Iloc = df.loc[:, :'Interview long']


In [51]:
i = dict(zip(df['Interview locations'], zip(df['Interview lat'], df['Interview long'])))
mf = dict(zip(df['Migrated From'], zip(df['Migrated From lat'], df['Migrated From long'])))
mt = dict(zip(df['Migrated To'], zip(df['Migrated To lat'], df['Migrated To long'])))
cr = dict(zip(df['Current Residence'], zip(df['Current Residence lat'], df['Current Residence long'])))

In [52]:
def cleaning_locations(df_dict, loc_dict):
    geoloc = Nominatim(user_agent="a")
    for k, v in df_dict.items():
        if k in loc_dict.keys():
            if v[0] != loc_dict[k][0] or v[1] != loc_dict[k][1]:
                print(k, df_dict[k], 'Bad Coordinates')
                location = geoloc.geocode(k)
                df_dict.update({k: (location.latitude, location.longitude)})
        else:
            print('Adding', k)
            location = geoloc.geocode(k)
            loc_dict.update({k: (location.latitude, location.longitude)})
            df_dict.update({k: (location.latitude, location.longitude)})

In [53]:
cleaning_locations(i, loc)
cleaning_locations(mf, loc)
cleaning_locations(mt, loc)
cleaning_locations(cr, loc)

Washington DC (40.22, -77.03) Bad Coordinates
Buffalo (34.05, -118.24) Bad Coordinates
Washington DC (23.4543, 45.6834) Bad Coordinates
Adding New York
Dallas (32.78, -96.79) Bad Coordinates
Adding Los Angeles 
Denver (39.73, -103.99) Bad Coordinates
Dallas (32.78, -96.79) Bad Coordinates
Los Angeles  (34.05, -118.24) Bad Coordinates
San Fran (39.5555, 34.2345) Bad Coordinates


In [59]:
dictionaries = [i, mf, mt, cr]
new_df = pd.DataFrame(columns=df.columns.to_list())
for d in dictionaries:
    df_temp = pd.DataFrame.from_dict(d, 'index')

ValueError: 12 columns passed, passed data had 2 columns

In [62]:
df_temp = pd.DataFrame.from_dict(i, 'index').reset_index(drop=False)
df_temp1 = pd.DataFrame.from_dict(mf, 'index').reset_index(drop=False)


,index,0,1
0,Los Angeles,34.050000,-118.240000
1,Dallas,32.770000,-96.790000
2,Washington DC,38.895037,-77.036543
3,Seattle,47.600000,-122.320000
4,New Jersey,40.140000,-74.670000
5,Denver,39.730000,-104.990000
6,Jackson,43.470000,-110.760000
7,Portland,45.510000,-122.670000
8,New York,40.712728,-74.006015


In [63]:
df_temp_all = pd.concat([df_temp, df_temp1], axis=1)
df_temp_all

,index,0,1,index,0,1
0,Denver,39.730000,-104.990000,Los Angeles,34.050000,-118.240000
1,New Jersey,40.140000,-74.670000,Dallas,32.770000,-96.790000
2,Washington DC,38.895037,-77.036543,Washington DC,38.895037,-77.036543
3,Seattle,47.600000,-122.320000,Seattle,47.600000,-122.320000
4,San Fran,37.730000,-122.460000,New Jersey,40.140000,-74.670000
5,New york,42.930000,-75.610000,Denver,39.730000,-104.990000
6,Santa Fe,35.690000,-105.930000,Jackson,43.470000,-110.760000
7,Dallas,32.770000,-96.790000,Portland,45.510000,-122.670000
8,Jackson,43.470000,-110.760000,New York,40.712728,-74.006015
9,Portland,45.510000,-122.670000,NaN,NaN,NaN


In [64]:
df

,Interview locations,Interview lat,Interview long,Migrated From,Migrated From lat,Migrated From long,Migrated To,Migrated To lat,Migrated To long,Current Residence,Current Residence lat,Current Residence long
0,Denver,39.73,-104.99,Los Angeles,34.0500,-118.2400,Dallas,32.78,-96.79,Dallas,32.7800,-96.7900
1,New Jersey,40.14,-74.67,Dallas,32.7700,-96.7900,Jackson,43.47,-110.76,Jackson,43.4700,-110.7600
2,Washington DC,40.22,-77.03,Washington DC,34.5623,12.7685,Portland,45.51,-122.67,Portland,45.5100,-122.6700
3,Seattle,47.60,-122.32,Seattle,47.6000,-122.3200,Los Angeles,34.05,-118.24,Los Angeles,34.0500,-118.2400
4,San Fran,37.73,-122.46,Washington DC,23.4543,45.6834,New Jersey,40.14,-74.67,New Jersey,40.1400,-74.6700
5,New york,42.93,-75.61,New Jersey,40.1400,-74.6700,Denver,39.73,-104.99,Denver,39.7300,-104.9900
6,Santa Fe,35.69,-105.93,Denver,39.7300,-104.9900,Denver,39.73,-104.99,Denver,39.7300,-120.9900
7,Dallas,32.77,-96.79,Jackson,43.4700,-110.7600,Denver,39.73,-104.99,Denver,39.7300,-104.9900
8,Jackson,43.47,-110.76,Portland,45.5100,-122.6700,Denver,39.73,-103.99,Denver,39.7300,-104.9900
9,Portland,45.51,-122.67,New York,41.9300,-75.6100,San Fran,37.73,-122.46,San Fran,39.5555,34.2345


Map final locations on a map and give different colors for incorrect and correct locational coordinates. 

Locations in df that are not in loc than geocode new city and add coordinates to loc df.